# Prep des macro-domaines DUFLT

Version février 2026 intégrant les 40 domaines de Denis

In [156]:
import pandas as pd
import os

import datetime

# date as "yyyymmdd"
suffix = datetime.datetime.now().strftime("%y%m%d")
suffix

'260213'

In [157]:
#filename = "Trevoux1743_paragraphs_260210.tsv"
filename = "Trevoux1743_html_260213.tsv"


df = pd.read_csv(os.path.join('..', 'data', '1743_LeRobert', filename), sep='\t', encoding='utf-8')
df.head()

,book,volume,numero,head,subEntryId,type,srcDomain,text,html
0,DUFLT_1743,1,1,A,1,mainEntry,NaN,Aest la première Lettre de l'Alphabet François...,A est la première Lettre de l'Alphabet Françoi...
1,DUFLT_1743,1,2,AAHUS,1,mainEntry,NaN,"AAHUS , s.Aahusium. Ville de l'Evéché de Munst...","AAHUS , s. <i>Aahusium.</i> Ville de l'Evéché ..."
2,DUFLT_1743,1,3,AAR,1,mainEntry,NaN,"AAR ,ou AHR. subst. Aara, Abrinca. Rivière d'A...","AAR , ou AHR. subst. <i>Aara, Abrinca.</i> Riv..."
3,DUFLT_1743,1,4,AAR,1,mainEntry,NaN,"AAR ,Arula ou Arola, & non pas Arosa, comme on...","AAR , Arula ou Arola, & non pas <i>Arosa,</i> ..."
4,DUFLT_1743,1,5,AARBRER,1,mainEntry,NaN,"AARBRER ,Terme ancien qui n'est pas aujourd'hu...","AARBRER , Terme ancien qui n'est pas aujourd'h..."


In [158]:
df.shape

(83413, 9)

In [159]:
df_domaines_denis = pd.read_excel('Conversion_DUFLT_protodesignants_MacroDomaines_Denis.xlsx')
df_domaines_denis.rename(columns={"subordinate_domain":"srcDomain", "Macro-domaines correspondants": "macroDomain"}, inplace=True)
df_domaines_denis = df_domaines_denis[["srcDomain", "macroDomain"]]

df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace('histoire', 'Histoire')
df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace(' Religion', 'Religion')
df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace('Métiers [ métal, minéral,  dérivés]', 'Métiers [métal, minéral, dérivés]')
df_domaines_denis['macroDomain'] = df_domaines_denis['macroDomain'].replace('Agriculture - Économie domestique', 'Agriculture - Économie rustique')

df_domaines_denis.head()

/Users/lmoncla/git/github.com/lmoncla/trevoux/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,srcDomain,macroDomain
0,Terme d'Agriculture,Agriculture - Économie rustique
1,Terme d'Agriculture & de Coutumes,Agriculture - Économie rustique
2,Terme d'Agriculture & de Jardinage,Agriculture - Économie rustique
3,Terme d'Agriculture & de labourage,Agriculture - Économie rustique
4,Terme d'Agriculture & de Vigneron,Agriculture - Économie rustique


In [160]:
print(df_domaines_denis['macroDomain'].unique())
print(len(df_domaines_denis['macroDomain'].unique()))

['Agriculture - Économie rustique' 'Anatomie' 'Antiquité' 'Architecture'
 'Arts mécaniques - Manufactures' 'Beaux-arts' 'Belles-lettres' 'Blason'
 'Chasse-pêche' 'Chimie' 'Commerce' 'Droit Jurisprudence' 'Géographie'
 'Grammaire' 'Histoire' 'Histoire naturelle' 'Jeu' 'Maréchalerie - Manège'
 'Marine' 'Mathématiques' 'Médailles' 'Médecine' 'Mesure'
 'Métiers [métal, minéral, dérivés]' 'Métiers [alimentation]'
 'Métiers [autres]' 'Métiers [bois]' 'Métiers [cuir, peaux]'
 'Métiers [papier]' 'Métiers [tissu, habit]' 'Militaire' 'Monnaie'
 'Musique' 'Pharmacie' 'Philosophie' 'Religion'
 'Sciences physico-mathématiques' 'Superstition' 'Voyage']
39


In [161]:
# lowercase src-domain columns
df_domaines_denis['srcDomain'] = df_domaines_denis['srcDomain'].str.lower()
df['srcDomain'] = df['srcDomain'].str.lower()

In [162]:
#TODO: improve matching by stripping accents, etc...
import unicodedata

def remove_accents(text):
    if not isinstance(text, str):
        return text
    # On normalise le texte en forme NFD (Normal Form Decomposition)
    # Cela sépare le caractère de son accent
    texte_normalise = unicodedata.normalize('NFD', text)
    
    # On filtre pour ne garder que les caractères qui ne sont pas des "marques" (accents)
    # Les caractères de type 'Mn' correspondent aux "Mark, Nonspacing"
    return "".join(c for c in texte_normalise if unicodedata.category(c) != 'Mn')

def remove_terme_de(text):
    if not isinstance(text, str):
        return text
    text = text.replace('terme de la ', '')
    text = text.replace("terme de l'", '')
    text = text.replace('terme des ', '')
    text = text.replace('terme de ', '')
    text = text.replace('terme du ', '')
    text = text.replace("terme d'", '')
    text = text.replace('termes de la ', '')
    text = text.replace("termes de l'", '')
    text = text.replace('termes des ', '')
    text = text.replace('termes de ', '')
    text = text.replace('termes du ', '')
    text = text.replace("termes d'", '')
    text = text.replace("termes", '')
    text = text.replace("terme", '')
    return text

df_domaines_denis['srcDomain'] = df_domaines_denis['srcDomain'].apply(remove_accents)
df['srcDomain'] = df['srcDomain'].apply(remove_accents)

df_domaines_denis['srcDomain'] = df_domaines_denis['srcDomain'].apply(remove_terme_de)
df['srcDomain'] = df['srcDomain'].apply(remove_terme_de)


In [163]:
# remove duplicates in df_domaines_denis
df_domaines_denis = df_domaines_denis.drop_duplicates(subset=['srcDomain'])

In [164]:
df = pd.merge(df, df_domaines_denis, on='srcDomain', how='left')
df.head()

,book,volume,numero,head,subEntryId,type,srcDomain,text,html,macroDomain
0,DUFLT_1743,1,1,A,1,mainEntry,NaN,Aest la première Lettre de l'Alphabet François...,A est la première Lettre de l'Alphabet Françoi...,NaN
1,DUFLT_1743,1,2,AAHUS,1,mainEntry,NaN,"AAHUS , s.Aahusium. Ville de l'Evéché de Munst...","AAHUS , s. <i>Aahusium.</i> Ville de l'Evéché ...",NaN
2,DUFLT_1743,1,3,AAR,1,mainEntry,NaN,"AAR ,ou AHR. subst. Aara, Abrinca. Rivière d'A...","AAR , ou AHR. subst. <i>Aara, Abrinca.</i> Riv...",NaN
3,DUFLT_1743,1,4,AAR,1,mainEntry,NaN,"AAR ,Arula ou Arola, & non pas Arosa, comme on...","AAR , Arula ou Arola, & non pas <i>Arosa,</i> ...",NaN
4,DUFLT_1743,1,5,AARBRER,1,mainEntry,NaN,"AARBRER ,Terme ancien qui n'est pas aujourd'hu...","AARBRER , Terme ancien qui n'est pas aujourd'h...",NaN


In [165]:
df.shape

(83413, 10)

In [153]:
# macro_domain not NaN
df[df['srcDomain'].notna()]

,book,volume,numero,head,subEntryId,type,srcDomain,text,html,macroDomain
15,DUFLT_1743,1,15,ABADIR,1,mainEntry,mythologie,"ABADIR ,ou ABADDIR ; car Priscien, qui nous a ...","ABADIR ,ou ABADDIR ; car Priscien, qui nous a ...",Antiquité
16,DUFLT_1743,1,16,ABAEUZ,1,mainEntry,coutume,"ABAEUZ , s. m. & adj.Terme de Coutume . Biens ...","ABAEUZ , s. m. & adj.Terme de Coutume . Biens ...",Droit Jurisprudence
17,DUFLT_1743,1,17,ABAJOUR,1,mainEntry,architecture,"ABAJOUR , s. m.Terme d'Architecture , Spiracul...","ABAJOUR , s. m.Terme d'Architecture , <i>Spira...",Architecture
18,DUFLT_1743,1,17,ABAJOUR,2,relatedEntry,botanique,"ABAJOUR , s. m.Abajour . s. m. Terme de Botani...",Abajour . s. m. Terme de Botanique . <i>Spirac...,Histoire naturelle
20,DUFLT_1743,1,19,ABAISSE,1,mainEntry,patissier,"ABAISSE , s. m.terme de Pâtissier . C'est la p...","ABAISSE , s. m.terme de Pâtissier . C'est la p...",Métiers [alimentation]
...,...,...,...,...,...,...,...,...,...,...
83385,DUFLT_1743,6,5560,ZULUFDGILER,1,mainEntry,relation,ZULUFDGILER . s. m.Terme de Relation . Enfant ...,ZULUFDGILER . s. m.Terme de Relation . Enfant ...,Voyage
83388,DUFLT_1743,6,5563,ZUNDANASTAN,1,mainEntry,relation,ZUNDANASTAN .Terme de Relation . C'est le livr...,ZUNDANASTAN .Terme de Relation . C'est le livr...,Voyage
83390,DUFLT_1743,6,5565,ZUNITTER,1,mainEntry,philosophie hermetique,"ZUNITTER ,ou ZITTER. s. m. Terme de Philosophi...","ZUNITTER ,ou ZITTER. s. m. Terme de Philosophi...",Philosophie
83392,DUFLT_1743,6,5567,ZURANDALE,1,mainEntry,fleuriste,ZURANDALE . s. f.Terme de Fleuriste . Nom de t...,ZURANDALE . s. f.Terme de Fleuriste . Nom de t...,Métiers [autres]


In [140]:
df[df['srcDomain'].notna() & ~df['macroDomain'].notna()]

,book,volume,numero,head,subEntryId,type,srcDomain,text,html,macroDomain
149,DUFLT_1743,1,108,ABIGEAT,1,mainEntry,droit romain,ABIGEAT Terme de Droit Romain . L'Abigeat est ...,ABIGEAT Terme de Droit Romain . L'<i>Abigeat</...,NaN
176,DUFLT_1743,1,129,ABOKELLE,1,mainEntry,negotiant en egypte & de relation,ABOKELLE Terme de Négotiant en Égypte & de Rel...,ABOKELLE Terme de Négotiant en Égypte & de Rel...,NaN
181,DUFLT_1743,1,132,ABOLITION,1,mainEntry,chancellerie,ABOLITION Terme de Chancellerie . Abolitio cri...,ABOLITION Terme de Chancellerie . <i>Abolitio ...,NaN
266,DUFLT_1743,1,182,ABREUVER,3,relatedEntry,vernisseur,"ABREUVER Abreuver , verbe actif, terme de Vern...","ABREUVER Abreuver , verbe actif, terme de Vern...",NaN
304,DUFLT_1743,1,208,ABSIDE,1,mainEntry,architecture & de liturgie,ABSIDE Absis & absida. Terme d'Architecture & ...,ABSIDE <i>Absis</i> & <i>absida.</i> Terme d'A...,NaN
...,...,...,...,...,...,...,...,...,...,...
80438,DUFLT_1743,6,4745,USER,9,relatedEntry,forets,"USER Usé, ée . part. pass. & adj. Habit usé, e...","USER Usé, ée . part. pass. & adj. Habit <i>usé...",NaN
80511,DUFLT_1743,6,4795,VÛE,16,relatedEntry,lettres de change,"VÛE Vûe , signifie aussi un simple regard. Asp...","VÛE Vûe , signifie aussi un simple regard. <i>...",NaN
80533,DUFLT_1743,6,4799,VUIDER,5,relatedEntry,artisans,"VUIDER Vuider , en termes d'Artisans , signifi...","VUIDER Vuider , en termes d'Artisans , signifi...",NaN
80538,DUFLT_1743,6,4800,"VULCAIN, VOLCAN",2,relatedEntry,hermetique,"VULCAIN, VOLCAN Le Vulcain des Philosophes , e...","VULCAIN, VOLCAN Le Vulcain des Philosophes , e...",NaN


In [166]:
# Liste des src-domain qui n'ont pas de macro-domain correspondant
l = list(df[df['srcDomain'].notna() & ~df['macroDomain'].notna()]['srcDomain'].unique())

# strip whitespace from values in l
l = [x.strip() for x in l]

# sort l: fist value without &, then values with & and sorted alphabetically
l = sorted(l, key=lambda x: (('&' in x), x))

print(len(l))

# put this as a first column in a csv file for manual annotation
pd.DataFrame(l, columns=['srcDomain']).to_excel(os.path.join('..', 'data', '1743_LeRobert', f'src_domains_without_macro_{suffix}.xlsx'), index=False)

506


In [167]:
path = os.path.join('..', 'data', '1743_LeRobert')
df.to_csv(os.path.join(path, "Trevoux1743_html_macro_"+suffix+".tsv"), sep="\t", index=False)
#df.to_excel(os.path.join(path, "Trevoux1743_html_macro_"+suffix+".xlsx"), index=False)

In [128]:
path = os.path.join('..', 'data', '1743_LeRobert')
df.to_csv(os.path.join(path, "Trevoux1743_paragraphs_macro_"+suffix+".tsv"), sep="\t", index=False)
df.to_excel(os.path.join(path, "Trevoux1743_paragraphs_macro_"+suffix+".xlsx"), index=False)

In [ ]:
df.shape